In [32]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn import set_config

from data_to_csv import df_to_csv
from utils import generate_description

df_0 = pd.read_csv(folder_path + '/SBA_Cleaned_0.csv')

In [ ]:
df_0.dropna(inplace=True)

In [ ]:
df_0.shape

In [33]:
generate_description(df_0)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[42, 62, 23, 81, 44-45]"
1,Term,int64,0,0.0,377,"[96, 300, 180, 72, 110]"
2,NoEmp,int64,0,0.0,499,"[2, 1, 3, 8]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,216,"[0, 5]"
5,RetainedJob,int64,0,0.0,323,"[0, 4, 3]"
6,FranchiseCode,int64,0,0.0,2,"[0, 1]"
7,UrbanRural,int64,0,0.0,3,"[2, 1]"
8,RevLineCr,object,0,0.0,2,"[N, Y]"
9,LowDoc,object,0,0.0,2,"[N, Y]"


# RandomForest avec * features du csv_clean_0

In [36]:
y = df_0['Approve']
X = df_0.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [37]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [39]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [40]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")

In [41]:
param_grid = {
    'model__n_estimators': [70],
    'model__max_depth': [25],
    'model__min_samples_split': [3],
    'model__min_samples_leaf': [80],
    'model__max_features': [None, 'sqrt', 'log2'],
    'model__max_samples': [0.7]
}

In [42]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'GrAppv', 'SBA_Appv'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         Index(['NAICS', 'RevLineCr', 'LowDoc'], dtype='object'))])),
                                       ('model',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             param_grid={'model__max_depth': [25],
                         'model__max_features': [None, 'sqrt', 'log2'],
                         'model__max_samples': [0.7],
                         'model__min_samples_leaf': [80],
                         'model__min_samples_split': [3],
                         'model__n_estimators': [70]},
             scoring='f1_macro')

In [43]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

Meilleurs hyperparamètres trouvés :
{'model__max_depth': 25, 'model__max_features': None, 'model__max_samples': 0.7, 'model__min_samples_leaf': 80, 'model__min_samples_split': 3, 'model__n_estimators': 70}
Meilleure précision trouvée : 0.897145952746096


In [44]:
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

In [45]:
f1_macro = f1_score(y_test, predictions, average='macro')
print("F1-score macro sur l'ensemble de test :", f1_macro)

F1-score macro sur l'ensemble de test : 0.898733003724068


# RandomForest Sans JobCreated & JobRetained

In [46]:
maybe_leaking_feature = ['RetainedJob', 'CreateJob']
df_0 = df_0.drop(maybe_leaking_feature, axis=1)

In [47]:
generate_description(df_0)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[62, 54, 31-33, 11, 81]"
1,Term,int64,0,0.0,377,"[240, 36, 205, 300, 84]"
2,NoEmp,int64,0,0.0,499,"[1, 8, 0, 4, 11]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,FranchiseCode,int64,0,0.0,2,"[1, 0]"
5,UrbanRural,int64,0,0.0,3,[1]
6,RevLineCr,object,0,0.0,2,"[N, Y]"
7,LowDoc,object,0,0.0,2,"[Y, N]"
8,GrAppv,int64,0,0.0,16880,"[210000, 116000, 25000, 238000, 1100000]"
9,SBA_Appv,int64,0,0.0,28240,"[120000, 106250, 10000, 12500, 223500]"


In [48]:
y = df_0['Approve']
X = df_0.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [49]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [50]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [51]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")

In [52]:
param_grid = {
    'model__n_estimators': [70],
    'model__max_depth': [25],
    'model__min_samples_split': [3],
    'model__min_samples_leaf': [80],
    'model__max_features': [None, 'sqrt', 'log2'],
    'model__max_samples': [0.7]
}

In [53]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

In [ ]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

Meilleurs hyperparamètres trouvés :
{'model__max_depth': 25, 'model__max_features': None, 'model__max_samples': 0.7, 'model__min_samples_leaf': 80, 'model__min_samples_split': 3, 'model__n_estimators': 70}
Meilleure précision trouvée : 0.9056260242709107


In [ ]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

In [ ]:
f1_macro = f1_score(y_test, predictions, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)

F1-score macro sur l'ensemble de test : 0.9054926582537084


# RandomForest sans * MAYBE feature

In [ ]:
maybe_leaking_fea = ['SBA_Appv', 'GrAppv']
df_0 = df_0.drop(maybe_leaking_fea, axis=1)

In [ ]:
generate_description(df_0)

In [ ]:
y = df_0['Approve']
X = df_0.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [ ]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [ ]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")

In [ ]:
param_grid = {
    'model__n_estimators': [70],
    'model__max_depth': [25],
    'model__min_samples_split': [3],
    'model__min_samples_leaf': [80],
    'model__max_features': [None, 'sqrt', 'log2'],
    'model__max_samples': [0.7]
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

In [ ]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

In [ ]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

In [ ]:
f1_macro = f1_score(y_test, predictions, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)